In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io

In [2]:
#TEST##
import requests
from bs4 import BeautifulSoup

url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("table")

print(table.find("tr"))

<tr class="xl7039" height="29" style="mso-height-source:userset;height:22.0pt">
<td align="left" class="xl7139" colspan="2" height="29" style="height:22.0pt;
  mso-ignore:colspan;width:213pt" width="284">My paper on equity risk premiums:</td>
<td class="xl7039" style="width:65pt" width="87"></td>
<td class="xl7039" style="width:65pt" width="87"></td>
<td class="xl7432" colspan="8" style="width:520pt" width="696"><a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4398884">https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4398884</a></td>
</tr>


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"

try:
    # Télécharger les infos du site
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.raise_for_status()

    # Rechercher la table
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'width': '919'})

    # Récupérer toutes les lignes
    trs = table.find_all('tr')

    # Trouver la ligne d'en-tête (celle qui contient 'Country')
    header_row = None
    for tr in trs:
        tds = tr.find_all('td')
        if tds and 'Country' in tds[0].get_text():
            header_row = tr
            break

    # Extraire les en-têtes
    headers = [td.get_text(strip=True) for td in header_row.find_all('td')]

    # Extraire les lignes
    rows = []
    start_index = trs.index(header_row) + 1
    for tr in trs[start_index:]:
        cols = [td.get_text(strip=True) for td in tr.find_all('td')]
        if len(cols) == len(headers):
            rows.append(cols)

except Exception as e:
    print("Erreur lors du scraping :", e)

# Créer le DataFrame
df_damodaran = pd.DataFrame(rows, columns=headers)




In [4]:
# --- 2. Nettoyer les colonnes numériques ---
 
for col in df_damodaran.columns[1:]:
    df_damodaran[col] = pd.to_numeric(df_damodaran[col].astype(str).str.replace('%','').str.replace(',',''), errors='coerce')

df_damodaran['Country'] = df_damodaran['Country'].str.replace('\n', ' ').str.strip() #Nettoie les colonnes contenant des noms de pays qui incluent des retours à la ligne

df_damodaran = df_damodaran.dropna(how='all') #Retire les lignes où toutes les cellules sont vides

print(df_damodaran.head())

                                   Country  \
0                                Abu Dhabi   
1                                  Albania   
2                                  Algeria   
3  Andorra               (Principality of)   
4                                   Angola   

   Adj. Default\n              Spread  Equity Risk\n              Premium  \
0                                0.49                                4.99   
1                                3.56                                9.13   
2                                2.98                                8.35   
3                                1.58                                6.46   
4                                6.44                               13.01   

   Country Risk\n              Premium  Corporate Tax\n              Rate  \
0                                 0.66                              15.00   
1                                 4.80                              15.00   
2                            

In [5]:
# --- 2. Exportation du CSV ---
df_damodaran.to_csv("country_risk_premiums.csv", index=False) #index=false empêche les indices de se retrouver dans le df